# Dataset

In [ ]:
from docs_jpmml_org import fetch_bank_marketing

dataset = fetch_bank_marketing()
#print(dataset)

X, y = dataset.getData()

cont_cols = dataset.getContinuousColumns()
cat_cols = dataset.getCategoricalColumns()

# Workflow

## Option A: ordinal encoding

Recommended for NumPy data.

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder

transformer = ColumnTransformer([
    ("cont", "passthrough", cont_cols),
    ("cat", OrdinalEncoder(), cat_cols)
])

# Identify categorical columns by index
#categorical_feature = [len(cont_cols) + categorical_idx for categorical_idx in range(0, len(cat_cols))]

# Identify categorical_columns by name
feature_name = cont_cols + cat_cols
categorical_feature = cat_cols

classifier = LGBMClassifier(random_state = 42)

pipeline = Pipeline([
    ("transformer", transformer),
    ("classifier", classifier)
])
#pipeline.fit(X, y, classifier__categorical_feature = categorical_feature)
pipeline.fit(X, y, classifier__feature_name = feature_name, classifier__categorical_feature = categorical_feature)

## Option B: casting

Recommended for Pandas data.

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn2pmml.preprocessing import CastTransformer

transformer = ColumnTransformer(
    [("cont", "passthrough", cont_cols)] +
    [(cat_col, CastTransformer("category"), [cat_col]) for cat_col in cat_cols]
)
transformer.set_output(transform = "pandas")

classifier = LGBMClassifier(random_state = 42)

pipeline = Pipeline([
    ("transformer", transformer),
    ("classifier", classifier)
])
pipeline.fit(X, y)

# Export to PMML

In [ ]:
classifier.booster_.save_model("Booster.txt")

In [ ]:
from sklearn2pmml import sklearn2pmml

sklearn2pmml(pipeline, "LightGBMPipeline.pmml")